In [9]:
from articlehelper import ArticleHandler, RePostprocessor, map_paragraphs, map_category_ukrpravda
import numpy as np
import pandas as pd
from clfhelpers import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, mutual_info_score, adjusted_mutual_info_score, accuracy_score
from analyzers import PosAnalyzer, PosFreqWordsAnalyzer, Morphology, PosLexAnalyzer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import mutual_info_classif
from sklearn.externals import joblib


class ClfFactoryPosLex:
    def __init__(self, analyzer = None, pwf_analyzer=PosFreqWordsAnalyzer(Morphology().getAnalyzer(), lemmatize_freq = True), 
                 lexngrams = None, ngramsize = 2):
        if analyzer is None:
            self.analyzer = PosLexAnalyzer(pwf_analyzer, lexngrams, ngramsize)
        else:
            self.analyzer = analyzer
    
    def get_analyzer(self):
        return self.analyzer.analyze
    
    def make_classifier(self):
        return MultinomialNB(alpha=0.05)
    
    def make_vectorizer(self):
        return TfidfVectorizer(tokenizer=self.get_analyzer(), ngram_range=(2, 4), min_df=10)
    
class GenreClassifier():
    def __init__(self, dataset='news'):
        '''
        dataset = {'news', 'bruk'}
        '''
        self.tfidf = None
        self.clf = None
        self.sel_perc = None       
        if dataset == 'news':
            self.load('saves\\clf_news_MNB_poslex.pkl', 'saves\\tfidf_news_MNB_poslex.pkl', 
                      'saves\\featsel_news_MNB_poslex.pkl')
        elif dataset == 'bruk':
            self.load('saves\\clf_bruk_MNB_poslex.pkl','saves\\tfidf_bruk_MNB_poslex.pkl') 
    
    def init(self):
        freq_words = pd.read_csv('data\\freq_words.txt', sep=' ', header=None, names=['word', 'freq'])['word'].values
        lexngrams = np.loadtxt('data\\news_bigrams.txt', dtype=object, encoding='utf-8')
        print('Frequent words count:', len(freq_words))
        print('Lexical ngrams count:', len(lexngrams))
        self.factory = ClfFactoryPosLex(None, PosFreqWordsAnalyzer(Morphology().getAnalyzer(), list(freq_words), lemmatize_freq=True), lexngrams)
        
    def train(self, X, y, percentile=0):
        self.tfidf = self.factory.make_vectorizer()
        self.clf = self.factory.make_classifier()
        self.sel_perc = SelectPercentile(mutual_info_classif, percentile) if percentile >= 1 else None
        vtrain = self.tfidf.fit_transform(X)        
        if self.sel_perc is not None:
            vtrain = self.sel_perc.fit_transform(vtrain, y)
        self.clf.fit(vtrain, y)
        
    def save(self, clf_name, tfidf_name, sel_perc_name=None):
        joblib.dump(self.clf, clf_name) 
        joblib.dump(self.tfidf, tfidf_name)
        if not self.sel_perc is None:
            joblib.dump(self.sel_perc, sel_perc_name)
            
    def load(self, clf_name, tfidf_name, sel_perc_name=None):
        self.clf = joblib.load(clf_name) 
        self.tfidf = joblib.load(tfidf_name)
        if not sel_perc_name is None:
            self.sel_perc = joblib.load(sel_perc_name)        
        
    
    def predict(self, raw_strings, predict_proba = False):
        vtest = self.tfidf.transform(raw_strings)
        if self.sel_perc is not None:
            vtest = self.sel_perc.transform(vtest)
        predictor = self.clf.predict_proba if predict_proba else self.clf.predict
        y_predicted = predictor(vtest.toarray())
        return y_predicted
    


In [ ]:
clf_news = GenreClassifier('news')
clf_bruk = GenreClassifier('bruk')

In [26]:
# https://gazeta.ua/blog/50347/prorochi-koshmari-putina-ukrayina-potribna-nato
s = '''НАТО підійшло до наших кордонів, відкриває Америку президент Росії. Ніби це трапилося вчора або в ніч на сьогодні. Ніби сусідньої з Калінінградською областю Польщі не прийняли туди ще зо два десятиріччя тому. Ніби "республіки радянської Прибалтики" Литва, Латвія та Естонія не стали повноправними членами "агресивного блоку" ще в березні 2004-го.
Куди він тоді дивився, президент Путін? Гаразд, коли приймали Польщу (а з нею не менш братні Чехію та Угорщину), президентом він ще не був. Його ще тільки вирощували в якійсь політтехнологічній колбі Кремля. Настоювали у спеціальних фізрозчинах під пильним наглядом прикремлівських франкенштайнів і готували світові несподіванку. Однак 2004-го він уже ого як кермував! І здається, на вступ до НАТО рідних "прибалтів" (а з ними ще чотирьох посткомуністичних країн із православною Болгарією включно) він тоді навіть не пискнув. Уявляєте – він, Путін, такий могутній і, як кажуть на Полтавщині, ловкий?'''
print(s[:300]+"...")
print(clf_news.predict([s]))
print(clf_bruk.predict([s]))

НАТО підійшло до наших кордонів, відкриває Америку президент Росії. Ніби це трапилося вчора або в ніч на сьогодні. Ніби сусідньої з Калінінградською областю Польщі не прийняли туди ще зо два десятиріччя тому. Ніби "республіки радянської Прибалтики" Литва, Латвія та Естонія не стали повноправними чле...
['blog']
['fiction']


In [27]:
# https://gazeta.ua/articles/avto/_podvijnij-nefart-tesla-na-avtopiloti-vrizavsya-v-policejske-avto/840337
s = '''У Каліфорнії автомобіль Tesla врізався у припарковане поліцейське авто.
На момент зіткнення електрокар рухався у режимі автопілота, повідомляє ВВС.
Водій зазнав незначних ушкоджень.'''
print(s[:300]+"...")
print(clf_news.predict([s]))
print(clf_bruk.predict([s]))

У Каліфорнії автомобіль Tesla врізався у припарковане поліцейське авто.
На момент зіткнення електрокар рухався у режимі автопілота, повідомляє ВВС.
Водій зазнав незначних ушкоджень....
['news']
['press']


In [28]:
# https://gazeta.ua/articles/celebrities/_oleg-vinnik-rozkriv-znachennya-svogo-prizvischa/839950
s = '''Співак року за версією премії "Золота жар-птиця" Олег Винник, прикрасив обкладинку журналу "Телегід"
Артист завершив роботу над двома новими альбомами, озвучив мультиплікаційного героя, презентував кліп і відправився у всеукраїнський гастрольний тур. Про свій сценічний імідж , різницю між творчістю і показухою співак розповів в відвертому інтерв'ю журналу "Телегід".'''
print(s[:300]+"...")
print(clf_news.predict([s]))
print(clf_bruk.predict([s]))

Співак року за версією премії "Золота жар-птиця" Олег Винник, прикрасив обкладинку журналу "Телегід"
Артист завершив роботу над двома новими альбомами, озвучив мультиплікаційного героя, презентував кліп і відправився у всеукраїнський гастрольний тур. Про свій сценічний імідж , різницю між творчістю ...
['articles']
['prof_science']


In [29]:
# https://gazeta.ua/articles/opinions-journal/_tilki-gliboko-vpevnena-u-svoyij-pravoti-lyudina-mozhe-vibachatisya-pershoyu/838669
s = '''У мене завжди поганий настрій. Роботи більше, ніж вільного часу, хвора спина і серцева недостатність. Але ніколи цього не показую і не поширюю свого настрою на інших.
Справедливість – опора слабких. Сильний її не шукає. Знає, що справедливості немає.
Коли слабких і сильних порівнюють, у других забирають частину свободи під виглядом справедливості. Виходить, правила для всіх одні, а люди – різні.'''
print(s[:300]+"...")
print(clf_news.predict([s]))
print(clf_bruk.predict([s]))

У мене завжди поганий настрій. Роботи більше, ніж вільного часу, хвора спина і серцева недостатність. Але ніколи цього не показую і не поширюю свого настрою на інших.
Справедливість – опора слабких. Сильний її не шукає. Знає, що справедливості немає.
Коли слабких і сильних порівнюють, у других забир...
['articles']
['fiction']


In [30]:
# https://znaj.ua/science/ce-ne-spam-inoplanetyany-spilkuyutsya-z-namy-cherez-elektronnu-poshtu
s = '''Це не спам: інопланетяни спілкуються з нами через електронну пошту
Уфологи заявили, що комп'ютерні віруси - теж справа рук гуманоїдів
В Америці "мисливці за НЛО" повідомили, що знайшли незвичайну активність прибульців в інтернеті.
За словами уфологів, інопланетяни можуть посилати людям повідомлення через електронну пошту. Співробітники Гавайського університету змогли з'ясувати, що прибульці неодноразово намагалися вийти на зв'язок з людьми через соцмережі або електронну пошту.'''
print(s[:300]+"...")
print(clf_news.predict([s]))
print(clf_bruk.predict([s]))

Це не спам: інопланетяни спілкуються з нами через електронну пошту
Уфологи заявили, що комп'ютерні віруси - теж справа рук гуманоїдів
В Америці "мисливці за НЛО" повідомили, що знайшли незвичайну активність прибульців в інтернеті.
За словами уфологів, інопланетяни можуть посилати людям повідомлення ...
['news']
['press']
